<a href="https://colab.research.google.com/github/Yashgaudani/demo/blob/main/Create_a_chat_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# Importing necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

# Example conversation pairs (you can expand this with more data)
conversation_pairs = [
    ('Hi', 'Hello'),
    ('How are you?', 'I am fine, thank you.'),
    ('What is your name?', 'I am a chatbot.'),
    ('Who created you?', 'I was created by [Your Name].')
]

# Preparing data
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

for pair in conversation_pairs:
    input_text, target_text = pair
    # Use '\t' as the start sequence character for targets
    # Use '\n' as the end sequence character for targets
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0

# Define an input sequence and process it
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

# Set up the decoder, using encoder_states as initial state
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=64,epochs=50, validation_split=0.2)

# Inference setup
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

# Function to decode sequence back into something readable
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.0

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = target_characters[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        states_value = [h, c]

    return decoded_sentence

# Example usage
input_sentence = input('Enter a sentence: ')
input_seq = np.zeros((1, max_encoder_seq_length, num_encoder_tokens), dtype='float32')
for t, char in enumerate(input_sentence):
    input_seq[0, t, input_token_index[char]] = 1.0

decoded_sentence = decode_sequence(input_seq)
print('Input sentence:', input_sentence)
print('Decoded sentence:', decoded_sentence)


Epoch 1/50
1/1 [==============================] - 5s 5s/step - loss: 1.6081 - accuracy: 0.0215 - val_loss: 3.2735 - val_accuracy: 0.0968
Epoch 2/50
1/1 [==============================] - 0s 112ms/step - loss: 1.5972 - accuracy: 0.0753 - val_loss: 3.2650 - val_accuracy: 0.1613
Epoch 3/50
1/1 [==============================] - 0s 105ms/step - loss: 1.5876 - accuracy: 0.0753 - val_loss: 3.2556 - val_accuracy: 0.1613
Epoch 4/50
1/1 [==============================] - 0s 102ms/step - loss: 1.5776 - accuracy: 0.0753 - val_loss: 3.2433 - val_accuracy: 0.1613
Epoch 5/50
1/1 [==============================] - 0s 107ms/step - loss: 1.5655 - accuracy: 0.0753 - val_loss: 3.2225 - val_accuracy: 0.1613
Epoch 6/50
1/1 [==============================] - 0s 121ms/step - loss: 1.5475 - accuracy: 0.0753 - val_loss: 3.1805 - val_accuracy: 0.1613
Epoch 7/50
1/1 [==============================] - 0s 121ms/step - loss: 1.5104 - accuracy: 0.0753 - val_loss: 3.2024 - val_accuracy: 0.1613
Epoch 8/50
1/1 [=======